# **Pong**

---

# Unity 2D: Camera, Canvas e Risoluzione

In un progetto 2D, non stiamo solo togliendo una dimensione; stiamo cambiando il modo in cui il motore grafico "guarda" il mondo. Il setup corretto della telecamera e dell'interfaccia è fondamentale per evitare che il gioco appaia distorto su schermi diversi.

### 1. La Camera 2D: Proiezione Ortografica

Se selezioni la **Main Camera**, noterai che la proprietà *Projection* è impostata su **Orthographic**.

* **Cos'è:** A differenza della proiezione *Perspective* (usata nel 3D), la camera ortografica elimina il senso di profondità. Gli oggetti non diventano più piccoli man mano che si allontanano.
* **Size (Dimensione):** È il parametro principale. Non "muovi" la telecamera avanti e indietro; aumenti o diminuisci il valore `Size` per decidere quanta porzione di mondo mostrare.

### 2. Il Canvas (La base della GUI)

Per Pong, avremo bisogno di mostrare il punteggio. Per farlo, serve un **Canvas**.

* **Creazione:** `Hierarchy > UI > Canvas`.
* **Visualizzazione:** Attiva la modalità **2D** nella scena. Vedrai un enorme rettangolo bianco che sovrasta il tuo livello: quello è lo spazio dove "disegnerai" il punteggio e i menu.

### 3. Aspect Ratio e Risoluzione

Il testo introduce un passaggio cruciale: impostare una risoluzione fissa (**640x360**) nel **Game Window**.

* **Perché?** Se lasci "Free Aspect", la visuale cambierà ogni volta che ridimensioni la finestra di Unity, rendendo impossibile bilanciare il gioco.
* **Concetto:** L'*Aspect Ratio* è il rapporto (es. 16:9), mentre la *Resolution* è il numero effettivo di pixel (640x360). Impostando una risoluzione fissa, hai il controllo totale su cosa vede il giocatore.

### 4. Il Canvas Scaler (Responsività)

Questo è il componente che decide come si comporta la UI se lo schermo cambia dimensione.

* **UI Scale Mode:** Va impostato su **Scale With Screen Size**.
* **Reference Resolution:** Deve corrispondere a quella del gioco (**640x360**).
* **Match (Width or Height):** Per un gioco in orizzontale (Landscape) come Pong, si imposta il valore a **1 (Height)**.
* *Perché?* In questo modo, se lo schermo diventa più largo, la UI si adatterà basandosi sull'altezza, mantenendo le proporzioni corrette degli elementi di gioco.


### 5. Organizzazione: Scene e Cartelle

Il testo conclude con una "best practice" di ordine:

1. Salva la scena come "Game".
2. Crea una cartella **Scenes** nel pannello *Project*.
3. Sposta la scena lì dentro.
*Regola del Mentore:* Un progetto disordinato porta a bug difficili da trovare. Mantieni sempre gli asset divisi per tipo (Scripts, Scenes, Prefabs).

### Nota del Mentore: La "Pixel Perfection"

In un progetto 2D, la coordinata Z degli oggetti (Player, Palla) tecnicamente non influenza la loro dimensione a schermo grazie alla camera ortografica. Tuttavia, è bene tenerli tutti a `Z = 0` per evitare problemi di ordine di visualizzazione (chi sta sopra chi).

---

# Creating the Paddle: Gerarchia e Fisica 2D

Per creare un oggetto di gioco professionale, non ci limitiamo a trascinare un'immagine nella scena. Usiamo una struttura a "matrioska" (Padre-Figlio) per separare la logica dalla visualizzazione.

### 1. La Struttura Hierarchy: Parent & Child

1. **Paddle (Parent):** È un oggetto vuoto (`Create Empty`) che funge da "centro di comando". È qui che metteremo lo script e i componenti fisici.
2. **Sprite (Child):** È un oggetto figlio del Paddle che contiene il **Sprite Renderer**.
* **Perché questa separazione?** Se vuoi ruotare o scalare la grafica (lo Sprite) senza influenzare come la fisica calcola i rimbalzi, avere un padre vuoto ti salva la vita. Se sposti il "Paddle" sull'asse Y, lo "Sprite" lo seguirà fedelmente.

### 2. La Matematica dei Pixel: Da 14x60 a 0.14x0.60

Questo è un punto fondamentale per capire il rapporto tra arte 2D e Unity.

* **Pixels Per Unit (PPU):** Di default, Unity imposta 100 pixel = 1 Unità (1 metro nel mondo di gioco).
* **Il Calcolo:** Se la tua immagine (Sprite) è larga **14 pixel** e alta **60 pixel**, in Unity le sue dimensioni reali saranno:
* Larghezza: 
* Altezza: 

* **Box Collider 2D:** Ecco perché, quando vai a regolare la dimensione del Collider per farlo aderire perfettamente alla racchetta, inserisci i valori **0.14** e **0.60**. In questo modo, l'area di collisione fisica combacia esattamente con i pixel disegnati dall'artista.

### 3. Inquadratura: Orthographic Size 3.2

Inizialmente la telecamera ha una dimensione (Size) di 5. Il testo suggerisce di abbassarla a **3.2**.

* **Cosa succede?** Più abbassi il numero della `Size`, più la telecamera "zooma" verso il centro.
* **Risultato:** Le racchette appaiono più grandi e l'area di gioco sembra più stretta e intensa, perfetta per un gioco arcade come Pong.

### 4. Lo Script e i Componenti Fisici

Hai creato un nuovo script chiamato `Paddle.cs` e lo hai trascinato sull'oggetto "Paddle". Ora la racchetta ha un "cervello". Ma per interagire con il mondo, servono altri due componenti:

1. **Box Collider 2D:** Definisce il perimetro solido. Ricorda di usare sempre la versione **2D** (quella con l'icona verde e il "2D" nel nome).
2. **Rigidbody 2D:** È il motore fisico.
* **Impostazione Cruciale:** Devi impostare **Gravity Scale = 0**.
* **Perché?** Se lasci la gravità a 1, appena premi Play la tua racchetta cadrà verso il basso e uscirà dallo schermo. In Pong, le racchette devono "galleggiare" e muoversi solo quando lo dici tu.

### Nota del Mentore: La Precisione dei Collider

Regolare il Collider a 0.14 e 0.60 non è solo una questione estetica. Se il collider fosse più grande della grafica, la pallina rimbalzerebbe "nell'aria" prima ancora di toccare la racchetta, dando al giocatore una sensazione di bug o scarsa precisione. **La precisione nei numeri è la base di un buon gameplay.**

### Il Concetto: La Gerarchia delle Trasformazioni

In Unity, le dimensioni finali di un oggetto nel mondo sono il risultato di una **moltiplicazione**:

$$\text{Dimensione Finale} = \text{Scale del Transform} \times \text{Size del Collider}$$

1. **Se lo Scale è 1:** Il Collider con Size 0.14 occupa esattamente 0.14 unità nel mondo. (Combacia con lo Sprite).
2. **Se porti lo Scale a 2:** Unity raddoppia **tutto** quello che c'è sull'oggetto. Lo Sprite raddoppia visivamente, e anche il Box Collider 2D (che ha Size 0.14) viene raddoppiato dal motore fisico, diventando "fisicamente" 0.28.

**L'errore comune:** Se tu scrivessi manualmente **0.28** nel campo Size del Collider mentre lo Scale è a **2**, otterresti un'area di collisione grande **0.56** (). La racchetta sarebbe invisibile per metà, ma la pallina ci rimbalzerebbe contro molto prima di toccarla!

---


# Vertical Movement: Muovere la Racchetta

Dopo aver configurato la parte fisica (Rigidbody e Collider), dobbiamo scrivere la logica che trasforma la pressione dei tasti in movimento reale.

### 1. Il Concetto di "Asse Verticale"

Unity utilizza un sistema chiamato **Input Manager** che raggruppa tasti diversi sotto nomi logici. L'asse `"Vertical"` è pre-configurato per rispondere a:

* Frecce direzionali (**Su / Giù**).
* Tasti **W** e **S**.
* Joystick analogici.

**Valori restituiti:**

* **1.0:** Tasto Su premuto completamente.
* **-1.0:** Tasto Giù premuto completamente.
* **0:** Nessun tasto premuto (o posizione neutra).

### Lo Script: `Paddle.cs`

Ecco come implementare il movimento in modo pulito ed efficiente, utilizzando la velocità del Rigidbody invece di spostare le coordinate manualmente.

```csharp
using UnityEngine;

public class Paddle : MonoBehaviour 
{
    // 1. Variabile PUBBLICA per la velocità
    // Apparirà nell'Inspector e potremo modificarla mentre il gioco gira.
    public float speed = 1f;

    void Update()
    {
        // 2. Lettura dell'Input
        // Restituisce un numero tra -1 e 1.
        float verticalMovement = Input.GetAxis("Vertical");

        /* 3. Applicazione della VELOCITÀ
           Accediamo al Rigidbody2D e modifichiamo la sua 'velocity'.
           Creiamo un nuovo Vector2:
           X = 0 (la racchetta non si muove a destra/sinistra)
           Y = input * velocità (movimento su/giù) */
        GetComponent<Rigidbody2D>().velocity = new Vector2(0, verticalMovement * speed);
    }
}

```

---

### Analisi Concettuale: Perché `velocity`?

#### 1. Fluidità (Smoothing)

`Input.GetAxis` non passa istantaneamente da 0 a 1. Ha una piccola "inerzia" (chiamata *Gravity* e *Sensitivity* nell'Input Manager). Questo fa sì che la racchetta non scatti bruscamente, ma acceleri e deceleri leggermente, rendendo il controllo più naturale.

#### 2. Forza vs Velocità

Invece di usare `AddForce` (che accumula spinta come un razzo), usiamo la **`velocity`**.

* Impostare la velocità direttamente è ideale per Pong perché vogliamo che la racchetta si muova a una velocità costante e si fermi immediatamente quando rilasciamo il tasto.

### Il Potere delle Variabili `public`

Il testo evidenzia un trucco fondamentale dello sviluppo in Unity:

* Dichiarando `public float speed = 1f;`, Unity crea un campo di testo nell'**Inspector**.
* Se la racchetta ti sembra lenta durante il test, non devi tornare nel codice: scrivi **3** o **5** nell'Inspector e riprova. Questo velocizza enormemente il bilanciamento del gioco (*Playtesting*).

### Nota del Mentore: Ottimizzazione `GetComponent`

Nello snippet del testo viene usato `GetComponent<Rigidbody2D>()` direttamente dentro `Update`.
**Consiglio tecnico:** Siccome `Update` viene eseguito decine di volte al secondo, chiamare `GetComponent` continuamente è "costoso" per la CPU.

---



# Screen Limits: Confini e Collisioni

Un gioco senza limiti permette agli oggetti di vagare all'infinito nel vuoto digitale. Per Pong, dobbiamo "inscatolare" l'area di gioco usando lo sfondo come riferimento visivo e i Collider come barriere invisibili.

### 1. Setup dello Sfondo e il "Trucco" Matematico

Il testo introduce un metodo rapido: trascinare l'immagine direttamente nella *Hierarchy*. Unity creerà automaticamente un oggetto con un `Sprite Renderer`.

**La sfida della risoluzione:**
Il tuo schermo è alto **360 pixel**, ma l'immagine dello sfondo è alta solo **320 pixel**.
Per farla combaciare perfettamente, usiamo la matematica direttamente nel campo *Scale* dell'Inspector:

* **Formula:** : $\text{Risoluzione Schermo} / \text{Altezza Immagine} = \text{Scale Factor}$ (Puoi farlo direttamente nell'inspector, Transform, digita la foruma 360/320)
* **Calcolo:** : $360 / 320 = \mathbf{1.125}$
* **Risultato:** Impostando lo **Scale Y a 1.125**, lo sfondo copre esattamente l'altezza del gioco. Lo Scale X viene portato a 2.25 per coprire la larghezza.

### 2. Creare i "Limit" (Barriere Invisibili)

Non vogliamo che la racchetta superi il bordo verde dello sfondo. Creiamo quindi degli oggetti "sentinella":

1. **Limit (Top):** Un oggetto vuoto posizionato a **Y = 1.51**.
2. **Limit (Bottom):** Una copia posizionata a **Y = -1.51**.

**Componenti necessari:**

* **Box Collider 2D:** Viene data una forma larga e sottile (Size X=5, Y=0.2) per coprire l'intera larghezza del campo. Questi collider fermeranno fisicamente la racchetta (che ha il suo Rigidbody e Collider).

### 3. Il concetto di "Tag" (Etichettatura)

Unity ha bisogno di un modo per distinguere un muro da un nemico o da una palla. Usiamo i **Tags**.

* **Cos'è un Tag:** È un'etichetta testuale che appiccichiamo a un GameObject.
* **Procedura:** Creiamo un nuovo Tag chiamato **"Limit"** e lo assegniamo a entrambi gli oggetti barriera.
* **Utilità futura:** Nel codice potremo scrivere: *"Se l'oggetto che ho colpito ha il tag 'Limit', allora fermati o rimbalza"*. È molto più efficiente che controllare il nome dell'oggetto.

### Analisi Concettuale: Perché 1.51?

Potresti chiederti perché i limiti sono a 1.51 e non a numeri tondi.

* Ricorda la formula .
* Se l'area di gioco è alta 3.6 unità (), il bordo superiore è a 1.8 ().
* Posizionando il collider a 1.51, stiamo tenendo conto dello spessore visivo del bordo verde dello sfondo. Stiamo allineando la "fisica" alla "grafica".

Ecco un riassunto dei concetti tecnici trattati, focalizzato sulla distinzione tra la gestione visiva e quella fisica in un ambiente Unity 2D.

## Gestione dell'Ordine Visivo (Rendering)

Nel contesto 2D, l'ordine con cui gli oggetti vengono disegnati sullo schermo non dipende dalla loro posizione sull'asse Z, ma da un sistema gerarchico dedicato.

* **Sorting Layers:** Agiscono come categorie macroscopiche (Sfondo, Gameplay, UI). Unity disegna i layer seguendo l'ordine della lista: i layer in fondo alla lista appaiono sopra quelli in cima.
* **Order in Layer:** All'interno dello stesso Sorting Layer, questo valore numerico determina la precedenza. Un numero maggiore indica che l'oggetto verrà disegnato sopra gli altri.
* **Limiti dell'asse Z:** Sebbene l'asse Z possa influenzare l'ordine nel rendering prospettico, nella telecamera **Orthographic** è sconsigliato usarlo per scopi grafici poiché può causare imprecisioni nei calcoli delle trasparenze e problemi di allineamento con i sistemi fisici.

## Gestione della Solidità e Confini (Fisica)

Mentre i layer gestiscono ciò che il giocatore vede, i **Collider** gestiscono ciò che gli oggetti possono fare nello spazio.

* **Screen Limits:** L'uso di oggetti invisibili dotati di **Box Collider 2D** permette di creare un perimetro invalicabile. Questo impedisce a elementi come la racchetta di uscire dall'area visibile.
* **Tags:** Queste etichette testuali permettono di categorizzare gli oggetti. Assegnare il tag "Limit" alle barriere consente al codice di identificare immediatamente con cosa è avvenuto un urto, permettendo di differenziare il comportamento tra diversi tipi di collisione.

## Integrità del Gameplay e Edge Case Testing

Il collaudo dei limiti fisici rientra nel cosiddetto **Edge Case Testing**. Questa pratica consiste nel testare i confini estremi delle meccaniche (ad esempio, forzare il movimento della racchetta contro un bordo per un tempo prolungato) per assicurarsi che:

1. L'oggetto non attraversi la barriera per errori di calcolo fisico.
2. Le coordinate non crescano all'infinito, causando instabilità nel software.


## Il Principio di Separazione delle Responsabilità

La distinzione fondamentale in Unity risiede nella separazione tra logica visiva e logica fisica:

* Il **Sorting Layer** è una direttiva per la scheda video per stabilire la successione dei pixel.
* Il **Collider** è una direttiva per il processore per calcolare le interazioni materiali.

Risolvere un problema visivo (evitare che un oggetto sparisca dietro lo sfondo) non garantisce la risoluzione di un problema meccanico (evitare che l'oggetto esca dal campo). Entrambi i sistemi devono essere configurati correttamente affinché il gioco sia coerente.

---



## Creazione dell'Oggetto Ball

La procedura segue la struttura gerarchica già adottata per la racchetta:

1. **Parent Object:** Si crea un GameObject vuoto denominato "Ball" che funge da contenitore logico e fisico.
2. **Child Object:** Si trascina l'asset grafico della pallina all'interno dell'oggetto Ball, creando uno Sprite figlio.
3. **Allineamento:** È fondamentale che lo Sprite sia posizionato a (0, 0, 0) rispetto al padre, affinché il centro visivo coincida con il centro fisico dell'oggetto.

## Gestione della Visibilità: Z-Axis vs Sorting Layers

Il testo evidenzia un problema comune: la pallina inizialmente non è visibile perché posizionata dietro lo sfondo. Sebbene la modifica dell'asse Z (es. -0.27) possa risolvere il problema temporaneamente, la **best practice** in Unity 2D consiste nel mantenere la posizione Z a **0** per tutti gli oggetti e gestire la profondità visiva tramite il sistema di Sorting.

### Il Sistema di Ordinamento

Unity determina cosa mostrare davanti attraverso due parametri dello Sprite Renderer:

1. **Sorting Layer:** È la categoria principale. L'ordine di disegno segue l'elenco definito nel progetto: i layer posizionati più in basso nella lista vengono disegnati per ultimi (e quindi appaiono sopra gli altri).
2. **Order in Layer:** All'interno dello stesso layer, determina la priorità numerica. Un valore più alto (es. 10) viene disegnato sopra un valore più basso (es. 0).

### Implementazione Pratica

Per il progetto Pong, l'organizzazione ottimale suggerita è:

* **Background:** Order in Layer = 0 (Sullo sfondo).
* **Paddles:** Order in Layer = 5 (Davanti allo sfondo).
* **Ball:** Order in Layer = 10 (Sempre in primo piano).

Successivamente, viene creato un nuovo Sorting Layer dedicato chiamato **"Game"** per raggruppare gli elementi attivi del gameplay, garantendo che siano sempre renderizzati sopra il layer di default o dello sfondo.

## Fisica e Dimensionamento

Il dimensionamento del sistema fisico segue rigorosamente la risoluzione dei pixel dell'asset:

* **Sprite Size:** 16x16 pixel.
* **PPU (Pixels Per Unit):** 100.
* **Box Collider 2D:** La dimensione viene impostata a **0.16 x 0.16** unità di Unity per far combaciare perfettamente l'area di collisione con la grafica.

### Componenti Necessari

Per abilitare le interazioni dinamiche, all'oggetto Ball viene aggiunto:

1. **Box Collider 2D:** Per rilevare gli impatti con racchette e limiti del campo.
2. **Rigidbody 2D:** Per permettere alla pallina di muoversi e reagire alle leggi della fisica.

## Integrità della Scena

Mantenere la coordinata Z a zero non è solo una scelta estetica, ma evita conflitti con il motore fisico 2D. In Unity 2D, i collider operano idealmente sullo stesso piano; allontanarli sull'asse Z per correggere problemi di visualizzazione potrebbe portare a mancate collisioni o comportamenti imprevedibili della pallina durante i rimbalzi.

---

## Ball Movement: Fisica e Trigger

Per gestire il rimbalzo in modo personalizzato tramite codice, la pallina deve essere configurata per rilevare le sovrapposizioni senza subire l'attrito o la resistenza fisica standard di Unity.

1. **Gravity Scale:** Deve essere impostata a **0** sul componente `Rigidbody 2D` per evitare che la pallina cada verso il basso.
2. **Is Trigger:** L'opzione "Is Trigger" sul `Box Collider 2D` deve essere attivata (**true**). Questo permette alla pallina di "entrare" nei collider di muri e racchette, attivando il metodo `OnTriggerEnter2D` invece di rimbalzare meccanicamente.
3. **Velocità Iniziale:** All'avvio, viene assegnata una velocità che combina la variabile `speed` (asse Y) con una piccola spinta laterale (asse X) per avviare lo scambio.

## Lo Script: `Ball.cs`

Il codice utilizza il sistema di Tag per identificare con cosa è avvenuto l'urto e inverte selettivamente le componenti del vettore velocità.

```csharp
using UnityEngine;

public class Ball : MonoBehaviour 
{
    public float speed = 1f;
    private Rigidbody2D ballRigidbody;

    void Start()
    {
        // Caching del componente per ottimizzare le performance
        ballRigidbody = GetComponent<Rigidbody2D>();
        
        // Imposta la direzione iniziale (lievemente inclinata)
        ballRigidbody.velocity = new Vector2(0.2f, speed);
    }

    void OnTriggerEnter2D(Collider2D otherCollider) 
    {
        // 1. RIMBALZO SUI LIMITI (Muri sopra/sotto)
        if (otherCollider.tag == "Limit") 
        {
            // Se urta il limite superiore e sta salendo -> inverti Y
            if (otherCollider.transform.position.y > transform.position.y && ballRigidbody.velocity.y > 0) 
            {
                ballRigidbody.velocity = new Vector2(ballRigidbody.velocity.x, -ballRigidbody.velocity.y);
            }
            // Se urta il limite inferiore e sta scendendo -> inverti Y
            if (otherCollider.transform.position.y < transform.position.y && ballRigidbody.velocity.y < 0) 
            {
                ballRigidbody.velocity = new Vector2(ballRigidbody.velocity.x, -ballRigidbody.velocity.y);
            }
        }
        // 2. RIMBALZO SULLE RACCHETTE (Paddles)
        else if (otherCollider.tag == "Paddle") 
        {
            // Se urta la racchetta sinistra e va a sinistra -> inverti X
            if (otherCollider.transform.position.x < transform.position.x && ballRigidbody.velocity.x < 0) 
            {
                ballRigidbody.velocity = new Vector2(-ballRigidbody.velocity.x, ballRigidbody.velocity.y);
            }
            // Se urta la racchetta destra e va a destra -> inverti X
            if (otherCollider.transform.position.x > transform.position.x && ballRigidbody.velocity.x > 0) 
            {
                ballRigidbody.velocity = new Vector2(-ballRigidbody.velocity.x, ballRigidbody.velocity.y);
            }
        }
    }
}

```

## Analisi Concettuale: La Riflessione del Vettore

L'algoritmo di rimbalzo implementato si basa sulla manipolazione dei segni del vettore velocità:

* **Collisione Orizzontale (Muri):** Si moltiplica `velocity.y` per . La pallina continua ad andare a destra o sinistra, ma cambia direzione verticale.
* **Collisione Verticale (Paddles):** Si moltiplica `velocity.x` per . La pallina continua a salire o scendere, ma cambia direzione orizzontale.

L'aggiunta dei controlli sulla direzione attuale della pallina (es. `ballRigidbody.velocity.x < 0`) è una misura di sicurezza fondamentale: assicura che il rimbalzo avvenga solo se la pallina si sta muovendo *verso* l'oggetto colpito, evitando che rimanga incastrata all'interno dei collider.

## Setup nell'Editor: Tagging

Affinché lo script funzioni, è necessario che gli oggetti nella Hierarchy siano correttamente etichettati:

1. Seleziona le racchette.
2. Nel menu a tendina **Tag** dell'Inspector, seleziona "Add Tag".
3. Crea il tag **"Paddle"** (rispettando le maiuscole) e assegnalo alle racchette.
4. Assicurati che i muri del campo abbiano il tag **"Limit"**.

## Nota del Mentore: RigidBody e IsTrigger

Hai notato che stiamo usando `OnTriggerEnter2D` invece di `OnCollisionEnter2D`.
In Unity, un **Trigger** è come un sensore laser: rileva il passaggio ma non oppone resistenza fisica. Questo ci permette di gestire il rimbalzo via codice con precisione assoluta, evitando che la pallina rallenti per l'attrito o che inizi a ruotare su se stessa a causa dei calcoli fisici del motore di gioco.

---

## Dynamically Adding the Ball: Prefab e GameController

Fino ad ora, la pallina era un oggetto "statico" già presente nella scena. Per creare un gioco scalabile e professionale, dobbiamo trasformarla in un **Prefab** (un modello riutilizzabile) e delegarne la creazione a un "cervello" centrale: il **GameController**.

### 1. Il Sistema dei Prefab: Lo "Stampino"

Un **Prefab** è essenzialmente un template salvato nei tuoi asset.

* **Creazione:** Trascinando l'oggetto `Ball` dalla Hierarchy alla cartella `Prefabs` nel pannello Project, crei un legame. Ora puoi cancellare la pallina dalla scena: la sua "matrice" è al sicuro tra gli asset.
* **Vantaggio:** Se modifichi il Prefab (ad esempio cambi il colore o la velocità), ogni istanza creata in futuro erediterà automaticamente quelle modifiche.

### 2. Il GameController: La Torre di Controllo

Il `GameController` è un oggetto vuoto che gestisce le regole globali del gioco. In questa fase, lo usiamo per organizzare la scena e far apparire la pallina all'avvio.

* **Gerarchia:** È buona norma rendere la Camera, lo Sfondo e i Paddle "figli" del `GameController` per mantenere la Hierarchy pulita e organizzata.
* **Posizionamento:** Anche se lavoriamo in 2D, Unity ragiona in tre dimensioni. Impostiamo la posizione del controller a  per assicurarci che la pallina nasca esattamente al centro del campo.

### Lo Script: `GameController.cs`

Questo script gestisce il riferimento al Prefab e la sua "nascita" fisica nel mondo di gioco.

```csharp
using UnityEngine;

public class GameController : MonoBehaviour 
{
    // 1. Riferimento al Prefab (da trascinare nell'Inspector)
    public GameObject ballPrefab;
    
    // Riferimento allo script 'Ball' che si trova sulla pallina istanziata
    private Ball currentBall;

    void Start () 
    {
        // 2. ISTANZIAZIONE
        // Crea una copia del prefab come figlio del GameController (transform)
        GameObject ballInstance = Instantiate(ballPrefab, transform);
        
        // 3. RECUPERO COMPONENTE
        // Prendiamo lo script 'Ball' per poterne gestire le proprietà in futuro
        currentBall = ballInstance.GetComponent<Ball>(); 
        
        // 4. POSIZIONAMENTO
        // Assicuriamoci che la pallina nasca al centro esatto
        currentBall.transform.position = Vector3.zero;
    }
}

```

### Analisi Tecnica: `Instantiate` e `Vector3.zero`

#### Il metodo `Instantiate`

È il comando che "stampa" l'oggetto nel gioco. Usando `Instantiate(ballPrefab, transform)`, diciamo a Unity due cose:

1. **Cosa creare:** Il `ballPrefab`.
2. **A chi appartiene:** Passando `transform`, la pallina diventa figlia del GameController, ereditandone la scala e la posizione relativa.

#### Perché `Vector3.zero`?

In Unity, `Vector3.zero` è una scorciatoia matematica per scrivere:
$$\text{Position} = (0, 0, 0)$$

Anche se il gioco è 2D, i componenti `Transform` richiedono sempre tre coordinate (). Usare `zero` garantisce che la pallina non sia "spostata" accidentalmente lungo l'asse Z, evitando i problemi di visibilità che abbiamo discusso in precedenza.

### Nota del Mentore: Il problema della monotonia

Al momento, la pallina viene creata dinamicamente, ma si muove sempre con lo stesso angolo predefinito nello script `Ball`. Un gioco di Pong dove la pallina parte sempre nello stesso modo sarebbe estremamente noioso e prevedibile.

---

# Random Velocity: Rendere il Gioco Imprevedibile

In un gioco come Pong, se la pallina partisse sempre con la stessa angolazione, il giocatore imparerebbe immediatamente dove posizionarsi, rendendo la sfida nulla. Per evitare questo, utilizziamo la classe `Random` di Unity per variare sia la **velocità** che la **direzione** iniziale.

### Lo Script Aggiornato: `Ball.cs`

Invece di un unico valore `speed`, ora definiamo dei range (minimo e massimo) per entrambi gli assi ( e ).

```csharp
using UnityEngine;

public class Ball : MonoBehaviour 
{
    // 1. Range di velocità configurabili dall'Inspector
    public float minXSpeed = 0.8f;
    public float maxXSpeed = 1.2f;
    public float minYSpeed = 0.8f;
    public float maxYSpeed = 1.2f;

    private Rigidbody2D ballRigidbody;

    void Start () 
    {
        ballRigidbody = GetComponent<Rigidbody2D>();

        // 2. Calcolo della velocità casuale
        // Random.Range sceglie un valore tra il minimo e il massimo.
        // (Random.value > 0.5f ? -1 : 1) decide se la direzione è positiva o negativa.
        ballRigidbody.velocity = new Vector2 (
            Random.Range(minXSpeed, maxXSpeed) * (Random.value > 0.5f ? -1 : 1),
            Random.Range(minYSpeed, maxYSpeed) * (Random.value > 0.5f ? -1 : 1)
        );
    }
}

```

### Analisi Concettuale: La Matematica del Caso

#### 1. `Random.Range(min, max)`

Questa funzione restituisce un numero decimale compreso tra i due estremi forniti. Ci permette di avere palline leggermente più lente o più veloci ad ogni "servizio", evitando la monotonia.

#### 2. L'Operatore Ternario (Il lancio della moneta)

La parte `(Random.value > 0.5f ? -1 : 1)` è un modo elegante per scrivere un `if/else` in una sola riga:

* `Random.value` genera un numero tra  e .
* Se il numero è maggiore di , usiamo **-1** (la pallina andrà a sinistra o in basso).
* Altrimenti usiamo **1** (la pallina andrà a destra o in alto).
Moltiplicando questo risultato per la velocità, decidiamo casualmente verso quale giocatore lanciare la sfida.

### Bilanciamento tramite Inspector

Poiché abbiamo dichiarato le variabili come `public`, ora il Ball Script nell'Inspector mostra quattro campi.

* **Polish & Tuning:** Se noti che la pallina parte con un'angolazione troppo "verticale" (rendendo difficile colpirla), puoi aumentare i valori di `minXSpeed` rispetto a `minYSpeed`.
* **Difficoltà:** Aumentando tutti i valori di "Max", il gioco diventerà molto più frenetico fin dal primo secondo.

### Nota del Mentore: La "Sensazione" di Gioco

L'uso del caso è ciò che trasforma un software deterministico in un "gioco". Tuttavia, troppa casualità può sembrare ingiusta.

---

# Scoring System: Interfaccia e Logica di Gioco

In questa fase, trasformiamo il semplice movimento fisico in una sfida competitiva. Implementiamo un sistema che rileva quando la pallina supera i limiti laterali del campo, assegna un punto al giocatore corretto e resetta il round.

### Creazione della UI (User Interface)

Per visualizzare il punteggio, utilizziamo il sistema **Unity UI**. Sebbene le versioni recenti suggeriscano *TextMeshPro*, questo tutorial utilizza il componente **Legacy Text** per semplicità didattica.

* **Struttura:** Gli oggetti di testo (**Score1** e **Score2**) vengono creati come figli del **Canvas**.
* **Posizionamento:** Vengono posizionati simmetricamente nella parte superiore dello schermo ().
* **Stile:** Viene applicato un font pixel-art e un colore verde per richiamare lo stile arcade classico.

### Lo Script: `GameController.cs`

Il `GameController` diventa il gestore dei dati. Deve conoscere il punteggio attuale, avere i riferimenti agli oggetti di testo per aggiornarli e decidere quando la pallina è "fuori gioco".

```csharp
using UnityEngine;
// 1. NAMESPACE NECESSARIO per gestire i componenti UI come 'Text'
using UnityEngine.UI;

public class GameController : MonoBehaviour 
{
    public GameObject ballPrefab;
    // Riferimenti agli oggetti di testo nella Hierarchy
    public Text score1Text;
    public Text score2Text;
    
    // Variabili per tracciare il punteggio numerico
    private int score1 = 0;
    private int score2 = 0;
    
    // Coordinata X oltre la quale viene assegnato il punto
    public float scoreCoordinates = 3.4f;
    
    private Ball currentBall;

    void Start () {
        SpawnBall();
    }

    // Metodo centralizzato per creare la pallina e resettare i testi
    void SpawnBall() {
        GameObject ballInstance = Instantiate(ballPrefab, transform);
        currentBall = ballInstance.GetComponent<Ball>();
        currentBall.transform.position = Vector3.zero;
        
        // Aggiorna la visualizzazione a schermo convertendo gli int in string
        score1Text.text = score1.ToString();
        score2Text.text = score2.ToString();
    }

    void Update () {
        if (currentBall != null) {
            // Se la pallina supera il limite destro (Punto per Giocatore 1)
            if (currentBall.transform.position.x > scoreCoordinates) {
                score1++;
                Destroy(currentBall.gameObject);
                SpawnBall();
            }
            // Se la pallina supera il limite sinistro (Punto per Giocatore 2)
            if (currentBall.transform.position.x < -scoreCoordinates) {
                score2++;
                Destroy(currentBall.gameObject);
                SpawnBall();
            }
        }
    }
}

```

### Analisi Concettuale: La "Linea di Porta"

#### 1. Rilevamento tramite Coordinate

Invece di usare dei collider fisici per i bordi laterali, lo script controlla costantemente la posizione  della pallina.

* Se , la pallina è "entrata in porta" a destra.
* Se , è uscita a sinistra.

#### 2. Gestione della Memoria (Garbage Collection)

Un passaggio fondamentale nel codice è l'uso di `Destroy(currentBall.gameObject)`.

* **Senza Destroy:** Ogni volta che viene segnato un punto, una nuova pallina verrebbe creata mentre quella vecchia continuerebbe a viaggiare nel vuoto infinito. Dopo pochi minuti, il gioco rallenterebbe drasticamente a causa delle migliaia di oggetti inutili.
* **Con Destroy:** Liberiamo la RAM eliminando l'oggetto vecchio prima di istanziarne uno nuovo tramite `SpawnBall()`.

#### 3. Il Metodo `ToString()`

Le variabili `score1` e `score2` sono numeri interi (`int`), ma il componente UI accetta solo stringhe di testo. Usiamo `.ToString()` per convertire il dato numerico in un formato che Unity possa "scrivere" sul Canvas.

### Nota del Mentore: Ottimizzazione della Logica

Nello script, il metodo `SpawnBall` aggiorna i testi ogni volta che la pallina nasce. Questo è un buon esempio di programmazione basata sugli eventi: non aggiorniamo il testo in ogni frame (sarebbe inutile e dispendioso), ma solo quando sappiamo che il punteggio è effettivamente cambiato.

**Domanda di ragionamento:**
Attualmente, se la pallina esce a destra, il punto va a `score1`. Considerando che in Pong il giocatore 1 è solitamente a sinistra e il giocatore 2 a destra, chi dovrebbe ricevere il punto quando la palla supera la coordinata **positiva** ()?

---


